In [76]:
import pandas as pd

In [79]:
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

--2024-05-20 13:38:02--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.160.37.133, 18.160.37.215, 18.160.37.115, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.160.37.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M   169MB/s    in 0.3s    

2024-05-20 13:38:03 (169 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]



In [177]:
df=pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [178]:
print(len(df.columns))

19


In [179]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [180]:
df['duration']=df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [181]:
df.duration =df.duration.apply(lambda td:td.total_seconds()/60)

In [182]:
df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [183]:
###########Q3. Dropping outliers
# Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

# What fraction of the records left after you dropped the outliers?

# 90%
# 92%
# 95%
# 98%

In [184]:
newdf=df[(df.duration>=1) & (df.duration<=60)]

In [185]:
left=len(newdf)/len(df)

In [186]:
left

0.9812202822125979

In [187]:
features=['PULocationID','DOLocationID']

In [188]:
newdf['PULocationID'] = newdf['PULocationID'].astype(str)
newdf['DOLocationID'] = newdf['DOLocationID'].astype(str)


/tmp/ipykernel_10897/432712770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['PULocationID'] = newdf['PULocationID'].astype(str)
/tmp/ipykernel_10897/432712770.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['DOLocationID'] = newdf['DOLocationID'].astype(str)


In [189]:
from sklearn.feature_extraction import DictVectorizer

In [190]:
train_dicts =newdf[features].to_dict(orient='records')

In [191]:
dv=DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [192]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [193]:
print(len(dv.feature_names_))

515


In [194]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [195]:
lr=LinearRegression()

In [196]:
target='duration'
y_train=newdf[target].values

In [197]:
lr.fit(X_train,y_train)

LinearRegression()

In [198]:
y_pred =lr.predict(X_train)

In [199]:
from sklearn.metrics import mean_squared_error

In [200]:
mean_squared_error(y_train,y_pred,squared=False)

7.649261027843963

In [201]:
df2=pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

In [202]:
df2['duration']=df2['tpep_dropoff_datetime'] - df2['tpep_pickup_datetime']

In [203]:
df2.duration =df2.duration.apply(lambda td:td.total_seconds()/60)

In [215]:
newdf2=df2[(df2.duration>=1) & (df2.duration<=60)]

In [216]:
features=['PULocationID','DOLocationID']

In [217]:
newdf2['PULocationID'] = newdf2['PULocationID'].astype(str)
newdf2['DOLocationID'] = newdf2['DOLocationID'].astype(str)

/tmp/ipykernel_10897/3737937636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf2['PULocationID'] = newdf2['PULocationID'].astype(str)
/tmp/ipykernel_10897/3737937636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf2['DOLocationID'] = newdf2['DOLocationID'].astype(str)


In [218]:
from sklearn.feature_extraction import DictVectorizer

In [219]:
train_dicts2 =newdf2[features].to_dict(orient='records')

In [220]:

X_train2 = dv.transform(train_dicts2)

In [221]:
print(len(dv.feature_names_))

515


In [222]:
target='duration'
y_train2=newdf2[target].values

In [223]:
from sklearn.metrics import mean_squared_error

In [224]:
y_pred2 =lr.predict(X_train2)

In [225]:
mean_squared_error(y_train2,y_pred2,squared=False)

7.811832556107575